In [ ]:
import tensorflow as tf 
import numpy as np 
import sys
from random import randint
import datetime
from sklearn.utils import shuffle
import pickle
import os

import getFacebookData
import training

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# Hyperparamters
batch_size = 24
max_encoder_len = 15
max_decoder_len = max_encoder_len
LSTM_units = 112
embedding_dim = lstmUnits
num_layers_LSTM = 3
num_iterations = 500000
word_vector_dim = 100 # Industry standard is 100-300

## Hidden facebook name for privacy
facebook_name = raw_input('Enter your full name: ')

## Create Dataset

In [8]:
messages_dict = {}

print 'Getting Facebook Data'
messages_dict.update(getFacebookData.read_data('fbmessages.txt',facebook_name))

print 'Total len of dictionary', len(messages_dict)

print 'Saving messages data dictionary'
np.save('messages_dictionary.npy', messages_dict)

Getting Facebook Data
Total len of dictionary 56603
Saving messages data dictionary


In [9]:
messages_file = open('messages_data.txt', 'w')
for key,value in messages_dict.iteritems():
    if (not key.strip() or not value.strip()):
        # If there are empty strings
        continue
    messages_file.write(key.strip() + value.strip())

In [11]:
full_msg_str, msg_data_dict = getFacebookData.process_data('messages_data.txt')
print 'Finished parsing and cleaning dataset'

words_list = msg_data_dict.keys()

Finished parsing and cleaning dataset


In [12]:
with open("words_list.txt", "wb") as fp: 
    pickle.dump(words_list, fp)

vocab_len = len(words_list)


In [13]:
pad_vector = np.zeros((1, word_vector_dim), dtype='int32')
EOS_vector = np.ones((1, word_vector_dim), dtype='int32')

In [14]:
words_list.append('<pad>')
words_list.append('<EOS>')
vocab_len = vocab_len + 2

## Create Training Matrices

In [ ]:
numTrainingExamples, xTrain, yTrain = training.create_matrix('messages_dictionary.npy', words_list, max_encoder_len)
# np.save('Seq2SeqXTrain.npy', xTrain)
# np.save('Seq2SeqYTrain.npy', yTrain)
# print 'Finished creating training matrices'

In [25]:
tf.reset_default_graph()

# Create the placeholders
encoder_inputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(max_encoder_len)]
decoder_labels = [tf.placeholder(tf.int32, shape=(None,)) for i in range(max_decoder_len)]
decoder_inputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(max_decoder_len)]
feed_previous = tf.placeholder(tf.bool)

In [26]:
encoder_LSTM = tf.contrib.rnn.BasicLSTMCell(lstmUnits, state_is_tuple=True)

In [27]:
decoder_outputs, decoder_final_state = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(encoder_inputs, 
                                                                                    decoder_inputs, 
                                                                                    encoder_LSTM, 
                                                                                    vocab_len, 
                                                                                    vocab_len, 
                                                                                    embedding_dim, 
                                                                                    feed_previous=feed_previous)

In [28]:
decoder_pred = tf.argmax(decoder_outputs, 2)

In [29]:
loss_weights = [tf.ones_like(l, dtype=tf.float32) for l in decoder_labels]
loss = tf.contrib.legacy_seq2seq.sequence_loss(decoder_outputs, decoder_labels, loss_weights, vocab_len)
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

## Training

In [30]:
sess = tf.Session()
saver = tf.train.Saver()

#saver.restore(sess, tf.train.latest_checkpoint('models/'))
sess.run(tf.global_variables_initializer())

In [31]:
# Uploading results to Tensorboard
tf.summary.scalar('Loss', loss)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [32]:
encoder_test_strings = ["whats up bro",
                        "hi",
                        "hey how are you",
                        "eh",
                        "i love you"
                     ]

In [33]:
zero_vector = np.zeros((1), dtype='int32')

In [34]:
## I interrupted the training at num_iterations = 100,000 because I am running this on AWS cloud and
## it was getting too costy to do 500,000 iterations.
## the 100,000 iterations took about 10 hours to run.
for i in range(num_iterations):

    encoder_train, decoder_target_train, decoder_input_train = training.get_batch(xTrain, yTrain, batch_size, max_encoder_len)
    feed_dict = {encoderInputs[t]: encoderTrain[t] for t in range(max_encoder_len)}
    feed_dict.update({decoder_labels[t]: decoder_target_train[t] for t in range(max_decoder_len)})
    feed_dict.update({decoder_inputs[t]: decoder_target_train[t] for t in range(max_decoder_len)})
    feed_dict.update({feed_previous: False})

    curr_loss, _, pred = sess.run([loss, optimizer, decoder_pred], feed_dict=feed_dict)
    
    if (i % 50 == 0):
        print('Current loss:', curr_loss, 'at iteration', i)
        summary = sess.run(merged, feed_dict=feed_dict)
        writer.add_summary(summary, i)
    if (i % 25 == 0 and i != 0):
        num = randint(0,len(encoder_test_strings) - 1)
        print encoder_test_strings[num]
        input_vector = training.get_test_input(encoder_test_strings[num], words_list, max_encoder_len);
        feed_dict = {encoder_inputs[t]: input_vector[t] for t in range(max_encoder_len)}
        feed_dict.update({decoder_labels[t]: zero_vector for t in range(max_decoder_len)})
        feed_dict.update({decoder_inputs[t]: zero_vector for t in range(max_decoder_len)})
        feed_dict.update({feed_previous: True})
        nums = (sess.run(decoder_pred, feed_dict=feed_dict))
        print training.nums_to_sentence(nums, words_list)

    if (i % 10000 == 0 and i != 0):
        save_path = saver.save(sess, "pretrained_seq2seq_new.ckpt", global_step=i)

('Current loss:', 11.792348, 'at iteration', 0)
i love you
[]
('Current loss:', 11.57716, 'at iteration', 50)
hi
[]
hey how are you
[]
('Current loss:', 10.117448, 'at iteration', 100)
hey how are you
[]
i love you
[]
('Current loss:', 8.3583117, 'at iteration', 150)
i love you
[]
whats up bro
[]
('Current loss:', 6.9422603, 'at iteration', 200)
whats up bro
[]
i love you
[]
('Current loss:', 5.9378028, 'at iteration', 250)
hey how are you
[]
i love you
[]
('Current loss:', 5.6092486, 'at iteration', 300)
hi
[]
hi
[]
('Current loss:', 4.7099671, 'at iteration', 350)
whats up bro
[]
whats up bro
[]
('Current loss:', 4.6967797, 'at iteration', 400)
whats up bro
[]
hi
[]
('Current loss:', 3.9221287, 'at iteration', 450)
i love you
[]
hi
[]
('Current loss:', 3.3816307, 'at iteration', 500)
eh
[]
hi
[]
('Current loss:', 3.6397047, 'at iteration', 550)
hey how are you
[]
hi
[]
('Current loss:', 3.7004254, 'at iteration', 600)
whats up bro
[]
hi
[]
('Current loss:', 3.3836544, 'at iteration',

['i ']
i love you
['i ']
('Current loss:', 2.6750174, 'at iteration', 5450)
eh
['studyyyyyynothing7t ']
i love you
['i ']
('Current loss:', 2.2419965, 'at iteration', 5500)
i love you
['i ']
whats up bro
['i ']
('Current loss:', 2.6137712, 'at iteration', 5550)
hey how are you
['i i ']
hi
['i ']
('Current loss:', 2.9715884, 'at iteration', 5600)
eh
['studyyyyyynothing7t ']
whats up bro
['i ']
('Current loss:', 2.854564, 'at iteration', 5650)
i love you
['i i ']
whats up bro
['i ']
('Current loss:', 2.9975364, 'at iteration', 5700)
hi
['i ']
hey how are you
['i ']
('Current loss:', 3.5855446, 'at iteration', 5750)
hi
['i ']
whats up bro
['i ']
('Current loss:', 2.6572368, 'at iteration', 5800)
i love you
['i ']
hey how are you
['i ']
('Current loss:', 2.6670597, 'at iteration', 5850)
i love you
['i i ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.8245268, 'at iteration', 5900)
hey how are you
['i ']
i love you
['i u ']
('Current loss:', 2.0507774, 'at iteration', 5950)
whats up bro


('Current loss:', 3.2116477, 'at iteration', 8550)
hey how are you
['i the the a a studyyyyyynothing7t ']
whats up bro
['i the the a a el el el el el el el el el el ']
('Current loss:', 2.7323663, 'at iteration', 8600)
whats up bro
['i the the a a el el el el el el el el el el ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.9806812, 'at iteration', 8650)
hi
['studyyyyyynothing7t ']
whats up bro
['i the the a a el el el el el el el el el el ']
('Current loss:', 2.4300549, 'at iteration', 8700)
i love you
['i the the a a studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.8467624, 'at iteration', 8750)
i love you
['i the the the the a studyyyyyynothing7t ']
i love you
['i the the the the the a studyyyyyynothing7t ']
('Current loss:', 2.8144958, 'at iteration', 8800)
hey how are you
['i the the the the the the studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.5651782, 'at iteration', 8850)
hey how are you
['i the the the the the a el el el el el 

['i not ']
hey how are you
['i not ']
('Current loss:', 2.3334451, 'at iteration', 11950)
hey how are you
['i not ']
i love you
['i not ']
('Current loss:', 2.1573579, 'at iteration', 12000)
eh
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.4499009, 'at iteration', 12050)
whats up bro
['studyyyyyynothing7t ']
i love you
['i not ']
('Current loss:', 2.6878328, 'at iteration', 12100)
hi
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.4101334, 'at iteration', 12150)
i love you
["i don't the the the the the the the the the the the the the "]
hey how are you
["i don't the the the the the the the the the the the the the "]
('Current loss:', 3.0684793, 'at iteration', 12200)
hi
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.4437466, 'at iteration', 12250)
hi
['studyyyyyynothing7t ']
hi
['studyyyyyynothing7t ']
('Current loss:', 2.4995048, 'at iteration', 12300)
hey how are you
["i don't the the a 

['studyyyyyynothing7t ']
i love you
["i don't have the call "]
('Current loss:', 2.8724625, 'at iteration', 15250)
i love you
["i don't have the call "]
i love you
["i don't have the call "]
('Current loss:', 2.3201685, 'at iteration', 15300)
i love you
["i don't have the call "]
hey how are you
["i don't have the call "]
('Current loss:', 2.482239, 'at iteration', 15350)
whats up bro
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.7651482, 'at iteration', 15400)
eh
['studyyyyyynothing7t ']
i love you
["i don't have the call "]
('Current loss:', 1.9744358, 'at iteration', 15450)
hey how are you
["i don't have the call "]
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.3757551, 'at iteration', 15500)
i love you
["i don't have the call "]
eh
['studyyyyyynothing7t ']
('Current loss:', 2.7087021, 'at iteration', 15550)
eh
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.2240136, 'at iteration', 15600)
hi
['studyyyyyynothi

...

["i don't want to ", 'is a lot of the same ']
i love you
["i don't want to do ", 'is a lot of ']
('Current loss:', 1.7111782, 'at iteration', 92300)
hi
['studyyyyyynothing7t ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 2.0505397, 'at iteration', 92350)
hey how are you
["i don't want to ", 'is a lot of the same ']
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
('Current loss:', 2.1282721, 'at iteration', 92400)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
eh
['studyyyyyynothing7t ', 'keda gedan ']
('Current loss:', 1.8569998, 'at iteration', 92450)
hey how are you
["i don't want to ", 'is a lot of the same ']
eh
['studyyyyyynothing7t ', 'keda ']
('Current loss:', 1.5163479, 'at iteration', 92500)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
hi
['studyyyyyynothing7t ']
('Current loss:', 2.0513661, 'at iteration', 92550)
eh
['studyyyyyynothing7t ', 'keda ']
what

['studyyyyyynothing7t ']
hi
['studyyyyyynothing7t ']
('Current loss:', 1.5371699, 'at iteration', 94850)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.6762389, 'at iteration', 94900)
eh
['studyyyyyynothing7t ', 'keda keda neik ']
eh
['studyyyyyynothing7t ', 'keda neik :d ']
('Current loss:', 1.3100452, 'at iteration', 94950)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
i love you
["i don't want to do ", 'is a lot of ']
('Current loss:', 1.5325246, 'at iteration', 95000)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
hi
['studyyyyyynothing7t ']
('Current loss:', 1.7192711, 'at iteration', 95050)
hey how are you
["i don't want to do ", 'is a lot of the same ']
i love you
["i don't want to do ", 'is a lot of ']
('Current loss:', 1.8142503, 'at iteration', 95100)
eh
['studyyyyyynothing7t ', 'keda neik :d ']
hi
['studyyyy

['studyyyyyynothing7t ', 'keda neik ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.569767, 'at iteration', 97500)
hey how are you
["i don't want to ", 'is a lot of the same ']
i love you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 2.2013175, 'at iteration', 97550)
eh
['studyyyyyynothing7t ', 'keda neik :d ']
whats up bro
['studyyyyyynothing7t ', 'is the same of the world ', 'is like ']
('Current loss:', 1.6387491, 'at iteration', 97600)
hey how are you
["i don't want to ", 'is a lot of the same ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.2038842, 'at iteration', 97650)
whats up bro
['studyyyyyynothing7t ', 'is the same of the world ']
eh
['studyyyyyynothing7t ', 'keda ']
('Current loss:', 1.6186496, 'at iteration', 97700)
whats up bro
['studyyyyyynothing7t ', 'is the same of the world ']
eh
['studyyyyyynothing7t ', 'keda neik :d ']
('Current loss:', 1.7169561, 'at iteration', 97750)
i lov

hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.627694, 'at iteration', 100150)
hi
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
('Current loss:', 1.4715137, 'at iteration', 100200)
hi
['studyyyyyynothing7t ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.8721842, 'at iteration', 100250)
hi
['studyyyyyynothing7t ']
hey how are you
["i don't want to ", 'to u ', 'is a email ']
('Current loss:', 1.5414333, 'at iteration', 100300)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
eh
['studyyyyyynothing7t el a5bar el studyyyyyynothing7t ']
('Current loss:', 1.9477507, 'at iteration', 100350)
hi
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ', 'keda neik ']
('Current loss:', 1.29482, 'at iteration', 100400)
hi
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ', 'keda neik ']
('Current loss:', 1.5842768, 'at iteration', 100450)
whats up br

KeyboardInterrupt: 